<a href="https://colab.research.google.com/github/sgoodfriend/rl-algo-impls/blob/cog2024/rl_algo_impls/microrts/colab_microrts_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Running Python MicroRTS submission in Google Colab
[sgoodfriend/rl-algo-impls](https://github.com/sgoodfriend/rl-algo-impls) was used to [train a Python agent to play MicroRTS](https://wandb.ai/sgoodfriend/rl-algo-impls-benchmarks/reports/rl-algo-impls-MicroRTS-Training--Vmlldzo0NjA2NTAy). [Farama-Foundation/MicroRTS-Py](https://github.com/Farama-Foundation/MicroRTS-Py) was the inspiration for this training.

Given MicroRTS is a Java environment, a Python server runs the Pytorch model, which communicates over a socket connection. This Colab notebook demonstrates what is necessary to install the necessary Python dependencies and expose a command-line endpoint for the submission jar file to communicate with.

## Steps to run RAIPerformanceTournament

1. Download the `rl_algo_impls-0.2.1.zip` submission file and unzip to reveal `RAISocketAI.jar` and `rl_algo_impls-0.2.1-py3-none-any.whl` Python installation file:

In [ ]:
%%capture
!apt-get install -y subversion
!wget https://github.com/sgoodfriend/rl-algo-impls/releases/download/v0.2.1/rl_algo_impls-0.2.1.zip
!unzip -j rl_algo_impls-0.2.1.zip

2. Install dependencies and rl_algo_impls Python wheel. This exposes the `rai_microrts` command-line endpoint, which will be called by the jar file. Ubuntu 20.04's default JDK is Java 11.

In [ ]:
# Fine if it prints errors as long as last line is "Successfully installed" with a long line of packages
!apt install -y default-jdk
!apt-get install swig
!python -m pip install --upgrade pip
!python -m pip install --upgrade torch
!python -m pip install --upgrade rl_algo_impls-0.2.1-py3-none-any.whl

3. Copy the jar files (microrts.jar and bot jars) and maps to run the tournament:

In [ ]:
%%capture
%%bash

# Get the package path
PACKAGE_PATH=$(pip show rl_algo_impls | grep '^Location:' | awk '{print $2 "/rl_algo_impls"}')
echo "The package is located at: $PACKAGE_PATH"

# Define the source directories
LIB_SOURCE="$PACKAGE_PATH/microrts/java/lib"
MAPS_SOURCE="$PACKAGE_PATH/microrts/java/maps"

# Define the target directories (current directory)
LIB_TARGET="./lib"
MAPS_TARGET="./maps"

# Remove existing directories if they exist
rm -rf "$LIB_TARGET" "$MAPS_TARGET"

# Copy directories
cp -r "$LIB_SOURCE" "$LIB_TARGET"
cp -r "$MAPS_SOURCE" "$MAPS_TARGET"

4. Run `tournaments.RAIPerformanceTournament` which runs 1 iteration of `RAIBCPPO`
   against 2 opponents on 3 maps of different sizes (16x16, 32x32, and 64x64). Depending on hardware, `RAIBCPPO` can average from 50ms+
   per turn on a Colab standard machine with 2-2.2GHz CPUs to 10-30ms per turn on modern
   CPUs, such as an Apple M1Max.

In [ ]:
!java -cp "RAISocketAI.jar:$(find lib -name "*.jar" | tr '\n' ':')" tournaments.RAIPerformanceTournament -m RAI-BC-PPO

## Performance Discussion for RAISocketAI (not RAIBCPPO)
This Colab notebook demonstrates that a GPU isn't required to run this agent. *However, if a GPU isn't available, a relatively beefy CPU is instead required for a model of this size to reliably compete under the 100ms turn timelimit.* The model has 32 convolutional layers and about 5 million parameters. So, beefy, but still smaller than something like a resnet50.

PyTorch uses CPU multithreading to speedup the inference, so CPU clock speed and available cores are both helpful. For example, here's the performance on hardware I've tested so far:

| CPU          | Threads | Clock Speed | 16x16 avg turn | 64x64 avg turn |
| -------------| --------| ------------| ---------------| ---------------|
| Colab        | 2       | 2.2 GHz     | 30-60 ms       | 450 ms         |
| Apple M1 Max | 8+2     | 3.2/2.2 GHz | 16 ms          | 60-65 ms       |
| Xeon 8358    | 30*     | 2.6 GHz     | 11 ms          | 25 ms          |

*CPU multithreading is capped to 16 threads in PyTorch

Occassionally, the agent will exceed 100 ms, even if the average is significantly below 100 ms. This happens more if there's other activity happening on the machine (for example, me using my own computer).

If the long computations are rare (less than once per game), a timeout on the Java side to submit an empty turn if the Python server doesn't respond within 100 ms is a good safety valve, though it could mean that some CPU cores are taken up by the Python server on the opponent's turn (killing the Python process instead would lead to considerable delay starting the server and loading the PyTorch model).

***However, if the competition machines are closer to the Colab machines, then a significantly smaller model is required.***